**jupyter中的事件循环EventLoop无法运行，所有调asyncio库的代码都无法运行  
根据报错推测，原因可能是jupyter本身就运行在一个asyncio的事件循环Eventloop中，在事件循环中无法再开启新的循环**

# 简介
asyncio是python3.4版本开始引入的，用于实现异步IO的内置标准库

python3.5开始，python增加了async/await关键字来替换原来定义协程的@asyncio.coroutine和yield from

python3.7开始，新增了asyncio.run()方法简化了原来的三步： 
1. 创建Eventloop
2. 往Eventloop中注册协程
3. 关闭Eventloop  

比如，原来的代码为：
``` python
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()
```
python3.7以后
``` python
asyncio.run(main())
```

由于不用创建Eventloop实例了，原本loop下的一些方法现在可以用asyncio调用  
比如python3.6: ```loop.create_task ``` -> python3.7: ```asyncio.create_task ```


## 异步IO
就是发起一个IO操作（如：网络请求，文件读写等），这些操作一般是比较耗时的，不用等待它结束，可以继续做其他事情，结束时会发来通知。

## 协程
又称为微线程，在一个线程中执行，执行函数时可以随时中断，由程序（用户）自身控制，执行效率极高，与多线程比较，没有切换线程的开销和多线程锁机制。 

## 协程与异步IO的关系
协程是异步io的一种实现手段


# gevent
gevent是第三方库，通过greenlet实现协程

In [2]:
import gevent

def funcA(t):
    print(1)
    gevent.sleep(t)
    print(3)

def funcB(t):
    print(2)
    gevent.sleep(t)
    print(4)
    
gevent.joinall([
    gevent.spawn(funcA, 1),
    gevent.spawn(funcB, 3)
])

1
2
3
4


[<Greenlet at 0x15fa676c9d8: _run>, <Greenlet at 0x15fa676cbf8: _run>]

# asyncio
asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持

## 基本概念
### Eventloop
Eventloop可以说是asyncio的核心，是中央总控。Eventloop实例提供了注册、取消和执行任务、回调等方法。

把一些异步函数(Coroutine，Future 或 Task)注册到这个事件循环上，事件循环会循环执行这些函数(但同时只能执行一个)，当执行到某个函数时，如果它正在等待I/O返回，事件循环会暂停它的执行去执行其他的函数；当某个函数完成I/O后会恢复，下次循环到它的时候继续执行。因此，这些异步函数可以协同(Cooperative)运行：这就是事件循环的目标

### Coroutine
协程(Coroutine)本质上是一个函数，特点是在代码块中可以将执行权交给其他协程


In [8]:
import asyncio


async def a():
    print('Suspending a')
    await asyncio.sleep(0)
    print('Resuming a')


async def b():
    print('In b')


async def main():
    await asyncio.gather(a(), b())


if __name__ == '__main__':
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

* asyncio.gather用来并发运行任务，在这里表示协同的执行a和b2个协程
* 在协程a中，有一句await asyncio.sleep(0)，await表示调用协程，sleep 0并不会真的sleep（因为时间为0），但是却可以把控制权交出去了。

### Future
Future代表了一个「未来」对象，异步操作结束后会把最终结果设置到这个Future对象上。  
Future是对协程的封装，不过日常开发基本不需要直接用这个底层Future类

可以对Future实例添加完成后的回调(add_done_callback)、取消任务(cancel)、设置最终结果(set_result)、设置异常(如果有的话，set_exception)等


In [6]:
import asyncio

def c():
    print('Inner C')
    return 12

loop = asyncio.get_event_loop()
future = loop.run_in_executor(None, c) 

print(type(future))
print("future.done():", future.done())

print("\n>> dir(future)")
for a in dir(future):
    if not a.startswith('_'):
        print(a)

<class '_asyncio.Future'>
future.done(): False

>> dir(future)
add_done_callback
cancel
cancelled
done
exception
get_loop
remove_done_callback
result
set_exception
set_result
Inner C


### Task
Eventloop除了支持注册协程Coroutine，还支持注册 Future 和 Task 2种类型的对象

* Future是协程的封装，Future对象提供了很多任务方法(如完成后的**回调**、**取消**、**设置任务结果**等等)
* Task是Future的子类，开发者并不需要直接操作Future这种底层对象，而是用Future的子类Task协同的调度协程以实现并发。

## 常用方法

### gather() vs wait()

比较gather和wait方法

```python
import time
import asyncio

async def a():
    print('Suspending a')
    await asyncio.sleep(3)
    print('Resuming a')


async def b():
    print('Suspending b')
    await asyncio.sleep(1)
    print('Resuming b')


async def f1():
    await a()
    await b()
    
async def f2():
    await asyncio.gather(a(), b())

s1 = f1()
s2 = f2()
s3 = asyncio.wait([a(), b()])

def main(s):
    st = time.time()
    asyncio.run(s)
    print(f'{s} Cost: {time.time() - st}\n')

for s in (s1,s2,s3):
    main(s)
```
执行结果
```
Suspending a
Resuming a
Suspending b
Resuming b
<coroutine object f1 at 0x7f4e724a5140> Cost: 4.0057666301727295

Suspending a
Suspending b
Resuming b
Resuming a
<coroutine object f2 at 0x7f4e71e10c40> Cost: 3.003472089767456

Suspending b
Suspending a
Resuming b
Resuming a
<coroutine object wait at 0x7f4e71db5740> Cost: 3.004014253616333
```

In [3]:
help(asyncio.gather)

Help on function gather in module asyncio.tasks:

gather(*coros_or_futures, loop=None, return_exceptions=False)
    Return a future aggregating results from the given coroutines/futures.
    
    Coroutines will be wrapped in a future and scheduled in the event
    loop. They will not necessarily be scheduled in the same order as
    passed in.
    
    All futures must share the same event loop.  If all the tasks are
    done successfully, the returned future's result is the list of
    results (in the order of the original sequence, not necessarily
    the order of results arrival).  If *return_exceptions* is True,
    exceptions in the tasks are treated the same as successful
    results, and gathered in the result list; otherwise, the first
    raised exception will be immediately propagated to the returned
    future.
    
    Cancellation: if the outer Future is cancelled, all children (that
    have not completed yet) are also cancelled.  If any child is
    cancelled, this is t

In [8]:
help(asyncio.wait)

Help on function wait in module asyncio.tasks:

wait(fs, *, loop=None, timeout=None, return_when='ALL_COMPLETED')
    Wait for the Futures and coroutines given by fs to complete.
    
    The sequence futures must not be empty.
    
    Coroutines will be wrapped in Tasks.
    
    Returns two sets of Future: (done, pending).
    
    Usage:
    
        done, pending = await asyncio.wait(fs)
    
    Note: This does not raise TimeoutError! Futures that aren't done
    when the timeout occurs are returned in the second set.

